In [ ]:
!rm -Rf HMP_Dataset
!git clone https://github.com/wchill/HMP_Dataset

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType

schema = StructType([
    StructField("x", IntegerType(), True),
    StructField("y", IntegerType(), True),
    StructField("z", IntegerType(), True)])

#df = spark.read.csv("user_click_seq.csv",header=False,schema=schema)


In [ ]:
import os

#get list of folders/files in folder HMP_Dataset
file_list = os.listdir('HMP_Dataset')

#filter list for folders containing data
file_list_filtered = [s for s in file_list if '_' in s]

In [ ]:
import os

#get list of folders/files in folder HMP_Dataset
file_list = os.listdir('HMP_Dataset')

#filter list for folders containing data
file_list_filtered = [s for s in file_list if '_' in s]

from pyspark.sql.functions import lit

#create pandas data frame for all the data

df = None

for category in file_list_filtered:
    data_files = os.listdir('HMP_Dataset/'+category)
    
    #create a temporary pandas data frame for each data file
    for data_file in data_files:
        print(data_file)
        temp_df = spark.read.option("header", "true").option("header", "false").option("delimiter", " ").csv('HMP_Dataset/'+category+'/'+data_file,schema=schema)
        
        #create a column called "source" storing the current CSV file
        temp_df = temp_df.withColumn("source", lit(data_file))
        
        #create a column called "class" storing the current data folder
        temp_df = temp_df.withColumn("class", lit(category))
        
        #append to existing data frame list
        #data_frames = data_frames + [temp_df]
                                                                                                             
        if df is None:
            df = temp_df
        else:
            df = df.union(temp_df)
        


In [ ]:
df.createOrReplaceTempView('df')
df_energy = spark.sql("""
select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label, class from df group by class
""")      
df_energy.createOrReplaceTempView('df_energy') 

In [ ]:
df_join = spark.sql('select * from df inner join df_energy on df.class=df_energy.class')

In [ ]:
splits = df_join.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [ ]:
df_train.count()

In [ ]:
df_test.count()

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)



In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)


In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, normalizer,lr])

model = pipeline.fit(df_train)

In [ ]:
model.stages[2].summary.r2

In [ ]:
model = pipeline.fit(df_test)

In [ ]:
model.stages[2].summary.r2